In [1]:
import nb_setup 
import importlib
importlib.reload(nb_setup)
nb_setup.init()

Added project root to sys.path: /Users/falcon/Developments/trading
Changed current working directory to: /Users/falcon/Developments/trading
Initialized project with base directory: /Users/falcon/Developments/trading


In [4]:
import pandas as pd
from settings import DATA_DIR

df = pd.read_csv(DATA_DIR / "all_ranked.csv", index_col=[0])
df.head()

,start_time,average_rank,rank_rolling_accumulated_pct_change,rank_rolling_variance_pct_change,open,close
symbol,,,,,,
BTC,2024-02-10 00:00:00,NaN,NaN,NaN,47166.0,47244.0
BTC,2024-02-10 00:15:00,46.0,46.0,NaN,47244.0,47204.0
BTC,2024-02-10 00:30:00,23.0,42.0,4.0,47204.0,47147.0
BTC,2024-02-10 00:45:00,18.0,34.0,2.0,47146.0,47116.0
BTC,2024-02-10 01:00:00,13.0,24.0,2.0,47122.0,47110.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 164217 entries, BTC to LDO
Data columns (total 6 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   start_time                           164217 non-null  object 
 1   average_rank                         164160 non-null  float64
 2   rank_rolling_accumulated_pct_change  164160 non-null  float64
 3   rank_rolling_variance_pct_change     164103 non-null  float64
 4   open                                 164217 non-null  float64
 5   close                                164217 non-null  float64
dtypes: float64(5), object(1)
memory usage: 8.8+ MB


In [12]:
import pandas as pd

def calculate_returns(df_final):
    # Initial principal amount
    principal = 100

    # Assuming df_final is your prepared DataFrame

    # Convert start_time to datetime if not already
    df_final['start_time'] = pd.to_datetime(df_final['start_time'])

    # Sort by start_time and average_rank to make sure we can pick top and bottom ranks easily
    df_final = df_final.sort_values(by=['start_time', 'average_rank'])

    # Find unique timestamps
    timestamps = df_final['start_time'].unique()

    # Initialize a list to store the results of each interval
    results = []

    for i in range(len(timestamps) - 1):
        current_time = timestamps[i]
        next_time = timestamps[i + 1]

        # Filter the DataFrame for the current timestamp
        current_interval = df_final[df_final['start_time'] == current_time]
        
        # Split into top 10 and bottom 10 coins
        top_10 = current_interval.head(10).copy()
        bottom_10 = current_interval.tail(10).copy()


        # Calculate the investment per coin
        investment_per_coin = principal / 20  # Divided among top 10 and bottom 10

        # Calculate returns for the next interval
        top_10['next_open'] = top_10['open'].shift(-1)
        top_10['next_close'] = top_10['close'].shift(-1)
        bottom_10['next_open'] = bottom_10['open'].shift(-1)
        bottom_10['next_close'] = bottom_10['close'].shift(-1)

        # Long the top 10
        long_returns = ((-top_10['next_open'] + top_10['next_close']) / top_10['next_open']) * investment_per_coin
        # Short the bottom 10
        short_returns = ((bottom_10['next_open'] - bottom_10['next_close']) / bottom_10['next_open']) * investment_per_coin

        # Sum returns and update principal
        interval_return = long_returns.sum() + short_returns.sum()
        principal += interval_return

        results.append({'timestamp': current_time, 'return': interval_return, 'principal': principal})
        print("---------------------------------------------")
        print(f"For interval {i}: Long {top_10.index.values}, Short {bottom_10.index.values}")
        print(f"In interval {i}, the return is {interval_return}, new principal is {principal}")

    # Convert the results to a DataFrame for easy analysis
    results_df = pd.DataFrame(results)

    # Calculate overall performance
    total_performance = results_df['return'].sum()

    print("Total simulated return over the period:", total_performance)

calculate_returns(df)

---------------------------------------------
For interval 0: Long ['BTC' 'ETH' 'SOL' 'WLD' 'WIF' 'DOGE' 'kPEPE' 'BNB' 'AVAX' 'NEAR'], Short ['GMT' 'MANTA' 'XAI' 'DYDX' 'BLUR' 'MAVIA' 'AAVE' 'LTC' 'CAKE' 'LDO']
In interval 0, the return is 0.34334005386886746, new principal is 100.34334005386887
---------------------------------------------
For interval 1: Long ['WIF' 'MINA' 'MAVIA' 'AVAX' 'ENS' 'BLUR' 'TIA' 'ALT' 'INJ' 'MATIC'], Short ['AI' 'XRP' 'GALA' 'AAVE' 'LINK' 'PYTH' 'XAI' 'DYM' 'SEI' 'STX']
In interval 1, the return is 0.5440976246313618, new principal is 100.88743767850023
---------------------------------------------
For interval 2: Long ['DYDX' 'PEOPLE' 'ADA' 'YGG' 'TIA' 'RUNE' 'kPEPE' 'FET' 'NFTI' 'SUI'], Short ['APT' 'FIL' 'DOT' 'ARB' 'ORDI' 'INJ' 'OP' 'SEI' 'AI' 'STX']
In interval 2, the return is 0.2692485120236243, new principal is 101.15668619052386
---------------------------------------------
For interval 3: Long ['RUNE' 'NFTI' 'SUI' 'LTC' 'SOL' 'RNDR' 'FTM' 'YGG' '